<a href="https://colab.research.google.com/github/petroDavydov/goit-DeepLearningForComputerVisionAndNLP/blob/main/dpnlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Topic 9. Data preparation in natural language processing***

Завантажимо всі необхідні бібліотеки

In [ ]:
import re
import string
import itertools
from collections import Counter

import pandas as pd
import numpy as np

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

import spacy

from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

from tqdm.auto import tqdm
tqdm.pandas()

import matplotlib.pyplot as plt
import seaborn as sns


Зчитаємо наш набір даних.

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DeepLearningforComputervisionandNLP/Reviews.csv', index_col='Id')


In [ ]:
df.head(3)

In [ ]:
df.info()


In [ ]:
df = df.loc[df['Score']!=3]

df.shape


In [ ]:
df['sentiment'] = [1 if score in [4, 5] else 0 for score in df['Score']]


Видалення дублікатів

In [ ]:
df.duplicated().sum()


Видалимо записи-дублікати:

In [ ]:
df = df.drop_duplicates().reset_index(drop=True)


Пошукаємо ідентичні відгуки про різні версії одного продукту

In [ ]:
df.groupby(['UserId', 'Time', 'Text']).count().sort_values('ProductId', ascending=False).head(10)


In [ ]:
search_text = "I have two cats, one 6 and one 2 years old. Both are indoor cats in excellent health. I saw the negative review and talked to my vet about it. I've also asked a number of veterinary professionals what to feed my cats and they all answer the same thing: Science Diet. Sure, you'll see stories of how one person's cat had issues, but even if that's 100% true, it's 1 case out of millions. Science and fact aren't based on someone's experience.<br /><br />So my point is, I love my cats and I'm very concerned about their health. I trust people who actually have medical degrees and experience with a wide range of animals. My only caution is do not fall for some hype or scare tactic that recommends some unproven or untested food or some fad diet for your pet. Don't listen to me, don't listen to the negative review. ASK YOUR VET what they recommend, and follow their instructions. My guess is you'll end up buying the Science Diet anyhow."
duplicates_example = df.loc[
    (df['UserId']=='A36JDIN9RAAIEC') &
    (df['Time']==1292976000) &
    (df['Text']==search_text)
]

duplicates_example


Видалимо однакові відгуки.

In [ ]:
df = df.drop_duplicates(subset={"UserId", "Time","Text"})

df.shape


# ***Нормалізація тексту (text normalization)***

Підготуємо допоміжні списки даних, які знадобляться нам для нормалізації.

In [ ]:
contractions = {
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}


Завантажимо перелік стоп-слів із бібліотеки nltk. Будемо використовувати структуру даних set() для більш швидкого доступу до даних.

In [ ]:
stop_words = set(stopwords.words('english')).union({'also', 'would', 'much', 'many'})


Видалимо слова-заперечення зі списку стоп-слів. Заперечення можуть мати ключову роль у визначенні сентименту. Так, якщо в реченні «I don’t like this product» ми видалимо заперечення «don’t», сентимент речення одразу перетвориться з негативного на позитивний.

In [ ]:
negations = {
    'aren',
    "aren't",
    'couldn',
    "couldn't",
    'didn',
    "didn't",
    'doesn',
    "doesn't",
    'don',
    "don't",
    'hadn',
    "hadn't",
    'hasn',
    "hasn't",
    'haven',
    "haven't",
    'isn',
    "isn't",
    'mightn',
    "mightn't",
    'mustn',
    "mustn't",
    'needn',
    "needn't",
    'no',
    'nor',
    'not',
    'shan',
    "shan't",
    'shouldn',
    "shouldn't",
    'wasn',
    "wasn't",
    'weren',
    "weren't",
    'won',
    "won't",
    'wouldn',
    "wouldn't"
}
stop_words = stop_words.difference(negations)


Для використання стемінгу завантажимо клас PorterStemmer з бібліотеки nltk.

In [ ]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()


# ***Регулярні вирази (regular expressions, RegEx)***

Завантажимо пайплайн бібліотеки spacy.

In [ ]:
nlp = spacy.load("en_core_web_sm", disable = ['parser','ner'])


Визначимо функцію нормалізації тексту.





In [ ]:
def normalize_text(raw_review):

    # Remove html tags
    text = re.sub("<[^>]*>", " ", raw_review) # match <> and everything in between. [^>] - match everything except >

    # Remove emails
    text = re.sub("\\S*@\\S*[\\s]+", " ", text) # match non-whitespace characters, @ and a whitespaces in the end

    # remove links
    text = re.sub("https?:\\/\\/.*?[\\s]+", " ", text) # match http, s - zero or once, //,
                                                    # any char 0-unlimited, whitespaces in the end

     # Convert to lower case, split into individual words
    text = text.lower().split()

    # Replace contractions with their full versions
    text = [contractions.get(word) if word in contractions else word
            for word in text]

    # Re-splitting for the correct stop-words extraction
    text = " ".join(text).split()

    # Remove stop words
    text = [word for word in text if not word in stop_words]

    text = " ".join(text)

    # Remove non-letters
    text = re.sub("[^a-zA-Z' ]", "", text) # match everything except letters and '

    # Stem words. Need to define porter stemmer above
    # text = [stemmer.stem(word) for word in text.split()]

    # Lemmatize words. Need to define lemmatizer above
    doc = nlp(text)
    text = " ".join([token.lemma_ for token in doc if len(token.lemma_) > 1 ])

    # Remove excesive whitespaces
    text = re.sub("[\\s]+", " ", text)

    # Join the words back into one string separated by space, and return the result.
    return text


Перевіримо її на тестовому прикладі.

In [ ]:
text = """On a quest for the perfedc1112t,,, !!!! <br />%%2%% popcorn to \ncompliment the Whirley Pop.  Don\'t get older, I\'m beginning to \nappreciate the more "natural" popcorn varieties, and I suppose\n that\'s what attracted me to the Arrowhead Mills Organic Yellow\n Popcorn.<br /> <br />I\'m no "organic" food expert.  I just wanted\n some good tasting popcorn.  And, I feel like that\'s what I got.\n  Using the Whirley Pop, with a very small amount of oil, I\'ve had \ngreat results.\n"""


print('Original text: ', text, '#'*30, sep='\n')

# -----------------
# normalized = normalize_text(text)

# words = normalized.split()
# chunks = [" ".join(words[i:i+9]) for i in range(0, len(words), 10)]

# formatted = "\n\n".join(chunks)

# print("\nNormalized text:\n", formatted, sep="")


print('\nNormalized text: ', normalize_text(text ), sep='\n')

Для демонстративних цілей зменшимо розмір набору даних до 5 тисяч прикладів

In [ ]:
df = df.groupby('sentiment').sample(2500, random_state=42)

df.shape


Застосуємо препроцесинг до тексту оглядів.

In [ ]:
df['text_normalized'] = df['Text'].progress_apply(normalize_text)


# ***Визначення метрики точності***

# ***Bag of Words***

Розділимо дані на навчальні й тестові.

In [ ]:
train_idxs = df.sample(frac=0.8, random_state=42).index
test_idxs = [idx for idx in df.index if idx not in train_idxs]
X_train = df.loc[train_idxs, 'text_normalized']
X_test = df.loc[test_idxs, 'text_normalized']

y_train = df.loc[train_idxs, 'sentiment']
y_test = df.loc[test_idxs, 'sentiment']


Створюємо й навчаємо об'єкт CountVectorizer

In [ ]:
vect = CountVectorizer().fit(X_train)

len(vect.vocabulary_)


Подивимось на приклад ознак, які було виокремлено. Тут ознакою буде кожен унікальний токен словника.

In [ ]:
vect.get_feature_names_out()[:5]


Перетворимо навчальні дані на матрицю документ-термін (document-term matrix).

In [ ]:
X_train_vectorized = vect.transform(X_train)
print(X_train_vectorized.shape)
X_train_vectorized


Створимо клас моделі й навчимо її на отриманих даних.

In [ ]:
model = LogisticRegression(random_state=42)
model.fit(X_train_vectorized, y_train)


Розрахуємо точність моделі.

In [ ]:
predictions = model.predict(vect.transform(X_test))

print('AUC: ', roc_auc_score(y_test, predictions))


Для подальшого швидкого тестування різних варіантів обробки даних напишемо допоміжну функцію.



* 👉🏻 Алгоритм tf-idf ми розглянемо в наступному підрозділі.*

In [ ]:
def get_preds(text_column, algorithm, ngrams=(1,1)):

    X_train = df.loc[train_idxs, text_column]
    X_test = df.loc[test_idxs, text_column]

    y_train = df.loc[train_idxs, 'sentiment']
    y_test = df.loc[test_idxs, 'sentiment']

    if algorithm == 'cv':
        vect = CountVectorizer(ngram_range=ngrams).fit(X_train)
    elif algorithm == 'tfidf':
        vect = TfidfVectorizer(ngram_range=ngrams).fit(X_train)
    else:
        raise ValueError('Select correct algorithm: `cv` or `tfidf`')

    print('Vocabulary length: ', len(vect.vocabulary_))

    # transform the documents in the training data to a document-term matrix

    X_train_vectorized = vect.transform(X_train)
    print('Document-term matrix shape:', X_train_vectorized.shape)

    model = LogisticRegression(random_state=42)
    model.fit(X_train_vectorized, y_train)

    predictions = model.predict(vect.transform(X_test))

    print('AUC: ', roc_auc_score(y_test, predictions))


Перевіримо результати моделювання на нетокенізованих даних.

In [ ]:
get_preds('Text', 'cv')


# ***TF-IDF***

Використаємо метод TF-IDF.

In [ ]:
get_preds('text_normalized', 'tfidf')


Для ненормалізованих даних:

In [ ]:
get_preds('Text', 'tfidf')


# ***N-Grams***

In [ ]:
get_preds('text_normalized', 'cv', (1,2))


In [ ]:
get_preds('text_normalized', 'tfidf', (1,2))


In [ ]:
get_preds('text_normalized', 'cv', (2,2))


In [ ]:
get_preds('Text', 'cv', (2,2))


In [ ]:
get_preds('Text', 'tfidf', (2,2))
